
# Conversion Sweep for Qubit Counts

This notebook benchmarks conversion routines between tableau, decision diagram (DD), matrix product state (MPS), and statevector (SV) representations across a sweep of qubit counts. For each qubit count, it generates a pseudo-random Clifford circuit so that all conversions remain valid, measures conversion times, and visualizes the results.


In [ ]:

from __future__ import annotations

import math
import time
from typing import Any, Dict, List

import numpy as np
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford
from qiskit_aer import AerSimulator

from quasar.conversion import dd_to_statevector, tableau_to_dd, tableau_to_statevector


In [ ]:

# Parameters for the sweep
qubit_counts = [4, 6, 8, 10, 12]
clifford_depth = 8
repeats_per_configuration = 3
base_seed = 1234

rng = np.random.default_rng(base_seed)
mps_backend = AerSimulator(method="matrix_product_state")


In [ ]:

def build_random_clifford_circuit(
    num_qubits: int,
    depth: int,
    circuit_rng: np.random.Generator,
    single_probability: float = 0.7,
    two_qubit_probability: float = 0.5,
) -> QuantumCircuit:
    """Construct a pseudo-random Clifford circuit compatible with all conversions."""

    if num_qubits <= 0:
        raise ValueError("Number of qubits must be positive")

    circuit = QuantumCircuit(num_qubits)
    layers = max(1, depth)

    single_pool = ("h", "s", "sdg", "x", "y", "z")
    for _ in range(layers):
        for q in range(num_qubits):
            if circuit_rng.random() < single_probability:
                gate_name = circuit_rng.choice(single_pool)
                getattr(circuit, gate_name)(q)

        indices = list(range(num_qubits))
        circuit_rng.shuffle(indices)
        for i in range(0, num_qubits - 1, 2):
            if circuit_rng.random() < two_qubit_probability:
                control, target = indices[i], indices[i + 1]
                if control == target:
                    continue
                if circuit_rng.random() < 0.5:
                    control, target = target, control
                circuit.cx(control, target)

    return circuit


def mps_to_statevector(mps_data: Any) -> np.ndarray:
    """Convert qiskit-aer's matrix product state payload into a dense statevector."""

    gammas, lambdas = mps_data
    num_qubits = len(gammas)
    dim = 1 << num_qubits
    state = np.zeros(dim, dtype=np.complex128)

    for index in range(dim):
        bits = format(index, f"0{num_qubits}b")
        amp = np.eye(1, dtype=np.complex128)
        for qubit, bit in enumerate(bits):
            gamma = gammas[qubit][int(bit)]
            amp = amp @ gamma
            if qubit < num_qubits - 1:
                lam = np.diag(np.asarray(lambdas[qubit], dtype=np.complex128))
                amp = amp @ lam
        state[index] = amp.squeeze()

    return state


def tableau_to_mps(
    clifford: Clifford,
    fallback_circuit: QuantumCircuit,
    backend: AerSimulator,
) -> Any:
    """Materialize an MPS representation for the provided tableau."""

    try:
        circuit = clifford.to_circuit()
        if not isinstance(circuit, QuantumCircuit):
            circuit = fallback_circuit
    except Exception:
        circuit = fallback_circuit

    qc = circuit.copy()
    qc.save_matrix_product_state()
    result = backend.run(qc).result()
    data = result.data(0)
    if "matrix_product_state" not in data:
        raise RuntimeError("Matrix product state payload missing from backend result")
    return data["matrix_product_state"]


In [ ]:

records: List[Dict[str, Any]] = []

for num_qubits in qubit_counts:
    for repeat in range(repeats_per_configuration):
        circuit_seed = int(rng.integers(0, np.iinfo(np.int32).max))
        local_rng = np.random.default_rng(circuit_seed)
        circuit = build_random_clifford_circuit(num_qubits, clifford_depth, local_rng)
        clifford = Clifford(circuit)

        record: Dict[str, Any] = {
            "num_qubits": num_qubits,
            "repeat": repeat,
            "circuit_seed": circuit_seed,
            "circuit_depth": int(circuit.depth()),
            "two_qubit_ops": int(circuit.num_nonlocal_gates()),
        }

        start = time.perf_counter()
        try:
            tableau_sv = tableau_to_statevector(clifford)
            record["tableau_to_sv_time_s"] = time.perf_counter() - start
            record["tableau_to_sv_success"] = True
            record["statevector_dimension"] = int(tableau_sv.size)
        except Exception as exc:
            record["tableau_to_sv_time_s"] = math.nan
            record["tableau_to_sv_success"] = False
            record["tableau_to_sv_error"] = repr(exc)
            tableau_sv = None

        start = time.perf_counter()
        try:
            dd_payload = tableau_to_dd(clifford)
            record["tableau_to_dd_time_s"] = time.perf_counter() - start
            record["tableau_to_dd_success"] = True
        except Exception as exc:
            record["tableau_to_dd_time_s"] = math.nan
            record["tableau_to_dd_success"] = False
            record["tableau_to_dd_error"] = repr(exc)
            dd_payload = None

        if dd_payload is not None:
            start = time.perf_counter()
            dd_sv = dd_to_statevector(dd_payload)
            record["dd_to_sv_time_s"] = time.perf_counter() - start if dd_sv is not None else math.nan
            record["dd_to_sv_success"] = dd_sv is not None
            if dd_sv is None:
                record["dd_to_sv_error"] = "Conversion returned None"
        else:
            record["dd_to_sv_time_s"] = math.nan
            record["dd_to_sv_success"] = False
            record["dd_to_sv_error"] = "Tableau->DD conversion failed"

        start = time.perf_counter()
        try:
            mps_payload = tableau_to_mps(clifford, circuit, mps_backend)
            record["tableau_to_mps_time_s"] = time.perf_counter() - start
            record["tableau_to_mps_success"] = True
        except Exception as exc:
            record["tableau_to_mps_time_s"] = math.nan
            record["tableau_to_mps_success"] = False
            record["tableau_to_mps_error"] = repr(exc)
            mps_payload = None

        if mps_payload is not None:
            start = time.perf_counter()
            try:
                mps_sv = mps_to_statevector(mps_payload)
                record["mps_to_sv_time_s"] = time.perf_counter() - start
                record["mps_to_sv_success"] = True
            except Exception as exc:
                record["mps_to_sv_time_s"] = math.nan
                record["mps_to_sv_success"] = False
                record["mps_to_sv_error"] = repr(exc)
        else:
            record["mps_to_sv_time_s"] = math.nan
            record["mps_to_sv_success"] = False
            record["mps_to_sv_error"] = "Tableau->MPS conversion failed"

        records.append(record)

raw_results = pd.DataFrame.from_records(records)
raw_results


In [ ]:

summary_columns = [
    "tableau_to_sv_time_s",
    "tableau_to_dd_time_s",
    "dd_to_sv_time_s",
    "tableau_to_mps_time_s",
    "mps_to_sv_time_s",
]

grouped = (
    raw_results.groupby("num_qubits", as_index=True)[summary_columns]
    .mean(numeric_only=True)
    .reset_index()
)
grouped


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
for column in summary_columns:
    plt.plot(
        grouped["num_qubits"],
        grouped[column],
        marker="o",
        label=column.replace("_time_s", "").replace("_", " -> "),
    )

plt.xlabel("Number of qubits")
plt.ylabel("Average conversion time (s)")
plt.title("Conversion runtimes vs. qubit count")
plt.legend()
plt.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()
